In [1]:
import pandas as pd
import os
import sys
import numpy as np
import wget
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

import sys

sys.path.append("scripts/")

import utils

import json

with open("settings.txt") as f:
    settings_dict = json.load(f)
np.random.seed(settings_dict["seed_choice"])

# sns_dict = settings_dict["sns_dict"]
# sns.set_theme(**sns_dict)

In [2]:
chen_cols = [
    "ID",
    "SourceID",
    "RAdeg",
    "DEdeg",
    "Per",
    "R21",
    "phi21",
    "T_0",
    "gmag",
    "rmag",
    "Per_g",
    "Per_r",
    "Num_g",
    "Num_r",
    "R21_g",
    "R21_r",
    "phi21_g",
    "phi21_r",
    "R^2_g",
    "R^2_r",
    "Amp_g",
    "Amp_r",
    "log(FAP_g)",
    "log(FAP_r)",
    "Type",
    "213-elta_min_g",
    "219-elta_min_r",
]

chenfeatures_df = pd.read_table(
    "data/Table2.txt",
    sep=" ",
    header=None,
    skipinitialspace=True,
    skiprows=34,
    names=chen_cols,
    index_col=1,
)

chenfeatures_df = chenfeatures_df.dropna()

# chenfeatures_df = chenfeatures_df.drop(
#     [
#         # Irrelevant:
#         "ID",
#         "RAdeg",
#         "DEdeg",
#         "T_0",
#         "Num_g",
#         "Num_r",
#         # Redundant:
#         "Per",
#         "Per_g",
#         "Per_r",
#         "rmag",
#         "gmag",
#         "Amp_g",
#         "Amp_r",
#         "phi21",
#         "R21",
#     ],
#     axis=1,
# )

class_df = chenfeatures_df[["Type"]]
chenfeatures_df = chenfeatures_df.drop(["Type"], axis=1)

In [3]:
all_classes = ["RSCVN", "BYDra", "DSCT", "EA", "EW", "RRc", "RR", "SR", "CEP", "Mira"]
class_df = class_df[class_df["Type"].isin(all_classes)]
class_df = (
    class_df.groupby("Type", group_keys=True)
    .apply(lambda x: x.sample(n=1200), include_groups=False)
    .reset_index()
    .set_index("SourceID")
)

chenfeatures_df = chenfeatures_df.loc[class_df.index]

In [4]:
class_df.value_counts()

Type 
BYDra    1200
CEP      1200
DSCT     1200
EA       1200
EW       1200
Mira     1200
RR       1200
RRc      1200
RSCVN    1200
SR       1200
Name: count, dtype: int64

In [5]:
chenfeatures_df.to_csv("data/chenfeatures_df.csv")

In [7]:
subcatalog = pd.read_csv("data/chenfeatures_df.csv", index_col=0)
oids = subcatalog.index.to_numpy()



In [13]:
import os
import shutil

dl_directory = "data/lightcurves/"
old_directory = (
    "/Users/sidchaini/Research/LCDistanceMetrics/phoenix/ztf_download/ztfdata/"
)


In [16]:
old_oids_files = os.listdir(old_directory)
old_oids = [
    filename.split(".")[0] for filename in old_oids_files if filename.endswith(".csv")
]

# Initialize lists to hold the categorized oids
oids_present = []
oids_absent = []

# Check each oid and categorize
for oid in tqdm(oids):
    if str(oid) in old_oids:
        oids_present.append(oid)
        # Copy the file to the current directory
        shutil.copy(os.path.join(old_directory, f"{oid}.csv"), dl_directory)
    else:
        oids_absent.append(oid)

  0%|          | 0/12000 [00:00<?, ?it/s]

In [20]:
len(oids_absent)

11243

In [11]:


# oids_present contains oids already in the old directory and copied over
# oids_absent contains oids not found in the old directory

array([770605,  19982,  29930, ..., 407848, 780022, 370338])